In [1]:
!pkill -f "Microsoft Excel"

In [2]:
import glob
import xlwings as xw
import pandas as pd
import itertools
import copy

In [6]:
xl_app = xw.App(visible=False)
wb = xl_app.books.open('./리스/backdata.xlsx')

In [7]:
data = wb.sheets['현대'].range('B1:AC2000').value
df = pd.DataFrame(data[1:], columns=data[0])
df = df.dropna(subset = 'brand_name')
df = df[df['판매여부'] != 'N']
wb.close()

print(df.columns)
df.head(3)

Index(['brand_name', 'brand_img', 'car_name', 'model_name', 'model_detial',
       'car_price', 'ext_color', 'int_color', 'is_limited', '판매여부',
       'brand_BNK', 'car_BNK', 'model_BNK', 'brand_MZ', 'car_MZ',
       'brand_woori', 'car_woori', 'brand_im', 'car_im', 'brand_lotte',
       'car_lotte', 'model_lotte', 'brand_sh', 'car_sh', 'brand_nh', 'car_nh',
       'model_nh', '전기차 보조금'],
      dtype='object')


,brand_name,brand_img,car_name,model_name,model_detial,car_price,ext_color,int_color,is_limited,판매여부,...,car_im,brand_lotte,car_lotte,model_lotte,brand_sh,car_sh,brand_nh,car_nh,model_nh,전기차 보조금
0,현대,https://www.carpan.co.kr/img/brand/1001.png,더 뉴 캐스퍼,2025년형 가솔린 1.0\n(2024.10~),스마트,14600000.0,"[['언블리치드 아이보리(NES)', 'background-color: rgb(18...","[['블랙', 'background-color: rgb(0, 0, 0);']]",None,None,...,캐스퍼 1.0,1.0,25.0,1.0,1.0,31.0,None,None,None,None
1,현대,https://www.carpan.co.kr/img/brand/1001.png,더 뉴 캐스퍼,2025년형 가솔린 1.0\n(2024.10~),디 에센셜,16800000.0,"[['선택 취소', 'background-color: rgb(182, 179, 17...","[['블랙', 'background-color: rgb(0, 0, 0);'], ['...",None,None,...,캐스퍼 1.0,1.0,25.0,1.0,1.0,31.0,None,None,None,None
2,현대,https://www.carpan.co.kr/img/brand/1001.png,더 뉴 캐스퍼,2025년형 가솔린 1.0\n(2024.10~),인스퍼레이션,19800000.0,"[['언블리치드 아이보리(NES)', 'background-color: rgb(18...","[['블랙', 'background-color: rgb(0, 0, 0);'], ['...",None,None,...,캐스퍼 1.0,1.0,25.0,1.0,1.0,31.0,None,None,None,None


In [8]:
def brand_idx(sh, x):
    x = x.strip().replace('\xa0', ' ')
    models = sh.range('J7', 'J36').value 
    models = [str(x).strip().replace('\xa0', ' ') for x in models]
    for idx, model in enumerate(models):
        if model == x:
            model_code = model[0]
            break
    return idx+1

def car_idx(sh,x):
    x = x.strip().replace('\xa0', ' ')
    models = sh.range('AM20', 'AM99').value 
    models = [str(x).strip().replace('\xa0', ' ') for x in models]
    for idx, model in enumerate(models):
        if model == x:
            model_code = model[0]
            break
    return idx+1

def car_model_idx(sh,x):
    x = str(x).strip().replace('\xa0', ' ')
    models = sh.range('AO20', 'AO256').value 
    models = [str(x).strip().replace('\xa0', ' ') for x in models]
    for idx, model in enumerate(models):
        if model == x:
            model_code = model[0]
            break
    return idx+1
    
def convert_params(provider, brand_name, car_name, model_name, car_price, org_brand_name, org_car_name, org_model_name, org_model_detail):
    if provider == 'bnk':
        input_data = {
            "bnk": {
                'brand_name': [0, 'B9', brand_name],
                'car_name': [0, 'B13', car_name],
                'model_name': [0, 'B15', model_name],
                'car_price': [1, 'N13', car_price],
                'deposit': [1, 'N36', 0],
                'downpayment': [1, 'N38', 0],
                'period': [0, 'B39', 0],
                'org_brand_name': org_brand_name,
                'org_car_name': org_car_name,
                'org_model_name': org_model_name,
                'org_model_detail': org_model_detail,
            },
        }
    if provider == 'sh':
        input_data = {
            "sh": {
                'brand_name': [0, 'B9', brand_name],
                'car_name': [1, 'AE7', car_name],
                'model_name': [1, 'AE7', car_name],
                'car_price': [1, 'AF8', car_price],
                'deposit': [1, 'AC35', 0],
                'downpayment': [1, 'AC36', 0],
                'period': [1, 'AC32', 0],
                'org_brand_name': org_brand_name,
                'org_car_name': org_car_name,
                'org_model_name': org_model_name,
                'org_model_detail': org_model_detail,
            },
        }
    if provider == 'mz':
        input_data = {
            "mz": {
                'brand_name': [0, 'AT2', brand_name],
                'car_name': [1, 'F5', car_name],
                'model_name': [2, 'F5', car_name],
                'car_price': [2, 'P9', car_price],
                'deposit': [2, 'AZ11', 0],
                'downpayment': [2, 'AZ16', 0],
                'period': [0, 'Z6', 0],
                'org_brand_name': org_brand_name,
                'org_car_name': org_car_name,
                'org_model_name': org_model_name,
                'org_model_detail': org_model_detail,
            },
        }
    if provider == 'lotte':
        input_data = {
            "lotte": {
                'brand_name': [0, 'N11', brand_name],
                'car_name': [0, 'P11', car_name],
                'model_name': [0, 'R11', model_name],
                'car_price': [1, 'BQ10', car_price],
                'deposit': [1, 'BK26', 0],
                'downpayment': [1, 'BD25', 0],
                'period': [1, 'BD24', 0],
                'org_brand_name': org_brand_name,
                'org_car_name': org_car_name,
                'org_model_name': org_model_name,
                'org_model_detail': org_model_detail,
            },
        }
    if provider == 'im':
        input_data = {
            "im": {
                'brand_name': [0, 'S7', brand_name],
                'car_name': [1, 'AS7', car_name],
                'model_name': [1, 'AS7', model_name],
                'car_price': [1, 'AS11', car_price],
                'deposit': [1, 'AS30', 0],
                'downpayment': [1, 'AS33', 0],
                'period': [1, 'AS28', 0],
                'org_brand_name': org_brand_name,
                'org_car_name': org_car_name,
                'org_model_name': org_model_name,
                'org_model_detail': org_model_detail,
            },
        }
    if provider == 'woori':
        input_data = {
            "woori": {
                'brand_name': [0, 'BA6', brand_name],
                'car_name': [0, 'BA7', car_name],
                'model_name': [0, 'BA7', model_name],
                'car_price': [0, 'BA12', car_price],
                'deposit': [0, 'BA43', 0],
                'downpayment': [0, 'BA41', 0],
                'period': [0, 'BA39', 0],
                'org_brand_name': org_brand_name,
                'org_car_name': org_car_name,
                'org_model_name': org_model_name,
                'org_model_detail': org_model_detail,
            },
        }
    if provider == 'nh':
        input_data = {
            "nh": {
                'brand_name' : [0, 'BT3', brand_name],
                'car_name' : [0, 'BT9', car_name],
                'model_name' : [0, 'BT11', model_name],        
                'car_price' : [0, 'AY9', car_price],
                'deposit' : [0, 'AY26', 0],
                'downpayment' : [0, 'AZ24', 0],
                'period' : [0, 'AY23', 0],
                'org_brand_name': org_brand_name,
                'org_car_name': org_car_name,
                'org_model_name': org_model_name,
                'org_model_detail': org_model_detail,
            },
        }

    return input_data  # 디버깅을 위해 input_data 출력

def convert_period(provider, x):
    if provider == 'bnk':
        if x == 1:
            return '60개월'
        if x == 2:
            return '48개월'
        if x == 3:
            return '36개월'
    if provider == 'sh':
        if x == 60:
            return '60개월'
        if x == 48:
            return '48개월'
        if x == 36:
            return '36개월'        
    if provider == 'mz':
        if x == 4:
            return '60개월'
        if x == 3:
            return '48개월'
        if x == 2:
            return '36개월'        
    if provider == 'lotte':
        if x == 60:
            return '60개월'
        if x == 48:
            return '48개월'
        if x == 36:
            return '36개월'        
    if provider == 'im':
        if x == 60:
            return '60개월'
        if x == 48:
            return '48개월'
        if x == 36:
            return '36개월'        
    if provider == 'woori':
        if x == 60:
            return '60개월'
        if x == 48:
            return '48개월'
        if x == 36:
            return '36개월'        
    if provider == 'nh':
        if x == 60:
            return '60개월'
        if x == 48:
            return '48개월'
        if x == 36:
            return '36개월'        


In [5]:
xl_app = xw.App(visible=True)
xl_app.display_alerts = False  # 경고 메시지 비활성화
wb = xl_app.books.open('./리스/견적기/bnk.xlsm')

In [7]:
sheet_names = ['Es1', 'Sheet1']
sheets = [wb.sheets[i] for i in sheet_names]
input_datum = df.apply(lambda x: convert_params('bnk', x['brand_BNK'], x['car_BNK'], x['model_BNK'], x['car_price'], x['brand_name'], x['car_name'], x['model_name'], x['model_detial']), axis=1).reset_index(drop = True)

In [8]:
deposits = [0, 0.1, 0.2, 0.3, 0.4]
downpayments = [0, 0.1, 0.2, 0.3, 0.4] 
periods = [1, 2, 3] # 60, 48, 36 
provider = 'bnk'

for idx, input_data in enumerate(input_datum):
    inputs = []  
    for combo in itertools.product(deposits, downpayments, periods):
        modified_input = copy.deepcopy(input_data)
        modified_input[provider]['deposit'][-1] = combo[0]
        modified_input[provider]['downpayment'][-1] = combo[1]
        modified_input[provider]['period'][-1] = combo[2]
        inputs.append(modified_input)

    output = []
    records = []
    for input_data in inputs:
        for i in input_data[provider].items():
            if 'org' not in i[0]:
                val = i[1][2]
                if i[0] == 'brand_name':
                    d = brand_idx(sheets[0], val)
                elif i[0] == 'car_name':
                    d = car_idx(sheets[0], val)
                elif i[0] == 'model_name':
                    d = car_model_idx(sheets[0], val)        
                else:
                    d = i[1][2]
                sheets[i[1][0]].range(i[1][1]).value = d 
        output.append(
            {
                "input" : input_data,
                "output": sheets[1].range('N51').value
            }
        )
        
    for entry in output:
        input_data = entry['input'][provider]
        output_value = entry['output']
        record = {
            'carpan_brand_name': input_data['org_brand_name'].strip().replace('\xa0', ' '),
            'carpan_car_name': input_data['org_car_name'].strip().replace('\xa0', ' '),
            'carpan_model_name': input_data['org_model_name'].strip().replace('\xa0', ' '),
            'carpan_model_detail': input_data['org_model_detail'].strip().replace('\xa0', ' '),
            'provider' : provider,
            'brand_name': input_data['brand_name'][2].strip().replace('\xa0', ' '),
            'car_name': input_data['car_name'][2].strip().replace('\xa0', ' '),
            'model_name': input_data['model_name'][2].strip().replace('\xa0', ' '),
            'car_price': input_data['car_price'][2],
            'deposit': input_data['deposit'][2],
            'downpayment': input_data['downpayment'][2],
            'period': input_data['period'][2],
            'output': output_value
        }
        records.append(record)
    df_out = pd.DataFrame(records)
    df_out['period'] = df_out['period'].apply(lambda x: convert_period(provider, x))
    df_out.to_csv(f"""./리스/결과/{record['provider']}_{record['carpan_brand_name']}_{record['carpan_car_name']}_{record['carpan_model_name'].replace('/', '')}_{idx}.csv""", encoding = 'cp949') 
    
    # if idx==10:
    #     break

KeyboardInterrupt: 

In [12]:
wb.close()

In [151]:
xl_app = xw.App(visible=True)
wb = xl_app.books.open('./리스/견적기/sh.xlsx')

In [152]:
sheet_names = ['리스모델', '리스_비교']
sheets = [wb.sheets[i] for i in sheet_names]
input_datum = df.apply(lambda x: convert_params('sh', x['brand_sh'], x['car_sh'], x['car_sh'], x['car_price'], x['brand_name'], x['car_name'], x['model_name'], x['model_detial']), axis=1)

In [153]:
def brand_idx(sheet1, x):
    if x == str:
        x = x.strip()
    if x == float:
        x = int(x)
        
    models = sheet1.range('B10', 'B20').value 
    models = [str(x).strip().replace('\xa0', ' ') for x in models]
    for idx, model in enumerate(models):
        if model == x:
            model_code = model[0]
            break
    return idx+1

def car_idx(sh,x):
    models = sh.range('C10', 'C100').value 
    return models[x]

In [154]:
deposits = [0, 0.1, 0.2, 0.3, 0.4]
downpayments = [0, 0.1, 0.2, 0.3, 0.4] 
periods = [36, 48, 60] # 60, 48, 36 
provider = 'sh'

for idx, input_data in enumerate(input_datum):
    inputs = []  
    for combo in itertools.product(deposits, downpayments, periods):
        modified_input = copy.deepcopy(input_data)
        modified_input[provider]['deposit'][-1] = combo[0] * 100
        modified_input[provider]['downpayment'][-1] = combo[1] * 100
        modified_input[provider]['period'][-1] = combo[2]
        inputs.append(modified_input)

    output = []
    records = []
    
    for input_data in inputs:
        for i in input_data[provider].items():
            if 'org' not in i[0]:
                val = i[1][2]
                if i[0] == 'brand_name':
                    d = int(val)
                elif i[0] == 'car_name':
                    car_name = car_idx(sheets[0], int(input_data['sh']['car_name'][2]))
                    d = car_name
                elif i[0] == 'model_name':
                    car_name = car_idx(sheets[0], int(input_data['sh']['car_name'][2]))     
                    d = car_name
                else:
                    d = i[1][2]
                sheets[i[1][0]].range(i[1][1]).value = d 
        output.append(
            {
                "input" : input_data,
                "output": sheets[1].range('K28').value
            }
        )
    for entry in output:
        input_data = entry['input'][provider]
        output_value = entry['output']
        record = {
            'carpan_brand_name': input_data['org_brand_name'].strip().replace('\xa0', ' '),
            'carpan_car_name': input_data['org_car_name'].strip().replace('\xa0', ' '),
            'carpan_model_name': input_data['org_model_name'].strip().replace('\xa0', ' '),
            'carpan_model_detail': input_data['org_model_detail'].strip().replace('\xa0', ' '),
            'provider' :provider,
            'brand_name': input_data['brand_name'][2],
            'car_name': car_name.replace('\xa0', ' '),
            'model_name': car_name.replace('\xa0', ' '),
            'car_price': input_data['car_price'][2],
            'deposit': input_data['deposit'][2] / 100,
            'downpayment': input_data['downpayment'][2] / 100,
            'period': input_data['period'][2],
            'output': output_value
        }
        records.append(record)
    df_out = pd.DataFrame(records)
    df_out['period'] = df_out['period'].apply(lambda x: convert_period(provider,x))
    df_out.to_csv(f"""./리스/결과/{record['provider']}_{record['carpan_brand_name']}_{record['carpan_car_name']}_{record['carpan_model_name'].replace('/', '')}_{idx}.csv""", encoding = 'cp949') 
    
    # if idx==10:
        # break

KeyboardInterrupt: 

In [ ]:
wb.close()

In [16]:
xl_app = xw.App(visible=True)
wb = xl_app.books.open('./리스/견적기/mz.xlsm')

In [18]:
sheet_names = ['리스수식', '차량정보', 'Sheet1']
sheets = [wb.sheets[i] for i in sheet_names]
input_datum = df.apply(lambda x: convert_params('mz', x['brand_MZ'], x['car_MZ'], x['car_MZ'], x['car_price'], x['brand_name'], x['car_name'], x['model_name'], x['model_detial']), axis=1)

In [19]:
deposits = [0, 0.1, 0.2, 0.3, 0.4]
downpayments = [0, 0.1, 0.2, 0.3, 0.4] 
periods = [4,3,2] # 60, 48, 36 
provider = 'mz'

for idx, input_data in enumerate(input_datum):
    inputs = []  
    for combo in itertools.product(deposits, downpayments, periods):
        modified_input = copy.deepcopy(input_data)
        modified_input[provider]['deposit'][-1] = combo[0]
        modified_input[provider]['downpayment'][-1] = combo[1] * input_data['mz']['car_price'][2]
        modified_input[provider]['period'][-1] = combo[2]
        inputs.append(modified_input)

    output = []
    records = []
    
    for input_data in inputs:
        for i in input_data[provider].items():
            if 'org' not in i[0]:
                val = i[1][2]
                if i[0] == 'brand_name':
                    d = int(val)
                elif i[0] == 'car_name':
                    d = val
                elif i[0] == 'model_name':
                    d = val        
                else:
                    d = i[1][2]
                sheets[i[1][0]].range(i[1][1]).value = d 
        output.append(
            {
                "input" : input_data,
                "output": sheets[2].range('AZ26').value
            }
        )
    for entry in output:
        input_data = entry['input'][provider]
        output_value = entry['output']
        record = {
            'carpan_brand_name': input_data['org_brand_name'].strip().replace('\xa0', ' '),
            'carpan_car_name': input_data['org_car_name'].strip().replace('\xa0', ' '),
            'carpan_model_name': input_data['org_model_name'].strip().replace('\xa0', ' '),
            'carpan_model_detail': input_data['org_model_detail'].strip().replace('\xa0', ' '),
            'provider' : provider,
            'brand_name': input_data['brand_name'][2],
            'car_name': input_data['car_name'][2],
            'model_name': input_data['model_name'][2],
            'car_price': input_data['car_price'][2],
            'deposit': input_data['deposit'][2],
            'downpayment': input_data['downpayment'][2] / input_data['car_price'][2],
            'period': input_data['period'][2],
            'output': output_value
        }
        records.append(record)
    df_out = pd.DataFrame(records)
    df_out['period'] = df_out['period'].apply(lambda x: convert_period(provider, x))
    df_out.to_csv(f"""./리스/결과/{record['provider']}_{record['carpan_brand_name']}_{record['carpan_car_name']}_{record['carpan_model_name'].replace('/', '')}_{idx}.csv""", encoding = 'cp949') 
    
    # if idx==10:
    #     break

KeyboardInterrupt: 

In [20]:
wb.close()

In [28]:
xl_app = xw.App(visible=True)
wb = xl_app.books.open('./리스/견적기/im.xlsm')

In [29]:
sheet_names = ['AG 입력시트', '운용리스_단일']
sheets = [wb.sheets[i] for i in sheet_names]
input_datum = df.apply(lambda x: convert_params('im', x['brand_im'], x['car_im'], x['car_im'], x['car_price'], x['brand_name'], x['car_name'], x['model_name'], x['model_detial']), axis=1)

In [31]:
deposits = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
downpayments = [0, 0.1, 0.2, 0.3, 0.4] 
periods = [60,48,36] # 60, 48, 36 
provider = 'im'

for idx, input_data in enumerate(input_datum):
    inputs = []  
    for combo in itertools.product(deposits, downpayments, periods):
        modified_input = copy.deepcopy(input_data)
        modified_input[provider]['deposit'][-1] = combo[0]
        modified_input[provider]['downpayment'][-1] = combo[1]
        modified_input[provider]['period'][-1] = combo[2]
        inputs.append(modified_input)

    output = []
    records = []
    
    for input_data in inputs:
        for i in input_data[provider].items():
            if 'org' not in i[0]:
                val = i[1][2]
                if i[0] == 'brand_name':
                    d = val
                elif i[0] == 'car_name':
                    d = val
                elif i[0] == 'model_name':
                    d = val        
                else:
                    d = i[1][2]
                sheets[i[1][0]].range(i[1][1]).value = d 
        output.append(
            {
                "input" : input_data,
                "output": sheets[1].range('AA27').value
            }
        )
    for entry in output:
        input_data = entry['input'][provider]
        output_value = entry['output']
        record = {
            'carpan_brand_name': input_data['org_brand_name'].strip().replace('\xa0', ' '),
            'carpan_car_name': input_data['org_car_name'].strip().replace('\xa0', ' '),
            'carpan_model_name': input_data['org_model_name'].strip().replace('\xa0', ' '),
            'carpan_model_detail': input_data['org_model_detail'].strip().replace('\xa0', ' '),
            'provider' : provider,
            'brand_name': input_data['brand_name'][2],
            'car_name': input_data['car_name'][2],
            'model_name': input_data['model_name'][2],
            'car_price': input_data['car_price'][2],
            'deposit': input_data['deposit'][2] ,
            'downpayment': input_data['downpayment'][2] ,
            'period': input_data['period'][2],
            'output': output_value
        }
        records.append(record)
    df_out = pd.DataFrame(records)
    df_out['period'] = df_out['period'].apply(lambda x: convert_period(provider, x))
    df_out.to_csv(f"""./리스/결과/{record['provider']}_{record['carpan_brand_name']}_{record['carpan_car_name']}_{record['carpan_model_name'].replace('/', '')}_{idx}.csv""", encoding = 'cp949') 
    
    # if idx==10:
    #     break

KeyboardInterrupt: 

In [27]:
wb.close()

In [155]:
xl_app = xw.App(visible=True)
wb = xl_app.books.open('./리스/견적기/lotte.xlsx')

In [156]:
sheet_names = ['DB관리', '운용리스_고잔가']
sheets = [wb.sheets[i] for i in sheet_names]
input_datum = df.apply(lambda x: convert_params('lotte', x['brand_lotte'], x['car_lotte'], x['model_lotte'], x['car_price'], x['brand_name'], x['car_name'], x['model_name'], x['model_detial']), axis=1)

In [157]:
deposits = [0, 0.1, 0.2, 0.3, 0.4]
downpayments = [0, 0.1, 0.2, 0.3, 0.4] 
periods = [60,48,36] # 60, 48, 36 
provider = 'lotte'

for idx, input_data in enumerate(input_datum):
    inputs = []  
    for combo in itertools.product(deposits, downpayments, periods):
        modified_input = copy.deepcopy(input_data)
        modified_input[provider]['deposit'][-1] = combo[0]
        modified_input[provider]['downpayment'][-1] = combo[1] * input_data[provider]['car_price'][2]
        modified_input[provider]['period'][-1] = combo[2]
        inputs.append(modified_input)

    output = []
    records = []
    
    for input_data in inputs:
        for i in input_data[provider].items():
            if 'org' not in i[0]:
                val = i[1][2]
                if i[0] == 'brand_name':
                    d = val
                elif i[0] == 'car_name':
                    d = val
                elif i[0] == 'model_name':
                    d = val        
                else:
                    d = i[1][2]
                sheets[i[1][0]].range(i[1][1]).value = d 
        output.append(
            {
                "input" : input_data,
                "output": sheets[1].range('AG14').value
            }
        )
    for entry in output:
        input_data = entry['input'][provider]
        output_value = entry['output']
        record = {
            'carpan_brand_name': input_data['org_brand_name'].strip().replace('\xa0', ' '),
            'carpan_car_name': input_data['org_car_name'].strip().replace('\xa0', ' '),
            'carpan_model_name': input_data['org_model_name'].strip().replace('\xa0', ' '),
            'carpan_model_detail': input_data['org_model_detail'].strip().replace('\xa0', ' '),
            'provider' : provider,
            'brand_name': input_data['brand_name'][2],
            'car_name': input_data['car_name'][2],
            'model_name': input_data['model_name'][2],
            'car_price': input_data['car_price'][2],
            'deposit': input_data['deposit'][2] ,
            'downpayment': input_data['downpayment'][2] / input_data['car_price'][2],
            'period': input_data['period'][2],
            'output': output_value
        }
        records.append(record)
    df_out = pd.DataFrame(records)
    df_out['period'] = df_out['period'].apply(lambda x: convert_period(provider, x))
    df_out.to_csv(f"""./리스/결과/{record['provider']}_{record['carpan_brand_name']}_{record['carpan_car_name']}_{record['carpan_model_name'].replace('/', '')}_{idx}.csv""", encoding = 'cp949') 
    
    # if idx==10:
    #     break

KeyboardInterrupt: 

In [ ]:
wb.close()

In [32]:
xl_app = xw.App(visible=True)
wb = xl_app.books.open('./리스/견적기/woori.xlsx')

In [33]:
sheet_names = ['운용리스']
sheets = [wb.sheets[i] for i in sheet_names]
input_datum = df.apply(lambda x: convert_params('woori', x['brand_woori'], x['car_woori'], x['car_woori'], x['car_price'], x['brand_name'], x['car_name'], x['model_name'], x['model_detial']), axis=1)

In [34]:
deposits = [0, 0.1, 0.2, 0.3, 0.4]
downpayments = [0, 0.1, 0.2, 0.3, 0.4] 
periods = [60,48,36] # 60, 48, 36 
provider = 'woori'


for idx, input_data in enumerate(input_datum):
    print(idx)
    if input_data['woori']['car_name'][2]:
        inputs = []  
        for combo in itertools.product(deposits, downpayments, periods):
            modified_input = copy.deepcopy(input_data)
            modified_input[provider]['deposit'][-1] = combo[0]
            modified_input[provider]['downpayment'][-1] = combo[1] * input_data[provider]['car_price'][2]
            modified_input[provider]['period'][-1] = combo[2]
            inputs.append(modified_input)
    
        output = []
        records = []
        
        for input_data in inputs:
            for i in input_data[provider].items():
                if 'org' not in i[0]:
                    val = i[1][2]
                    if i[0] == 'brand_name':
                        d = val
                    elif i[0] == 'car_name':
                        d = val
                    elif i[0] == 'model_name':
                        d = val        
                    else:
                        d = i[1][2]
                    sheets[i[1][0]].range(i[1][1]).value = d 
            output.append(
                {
                    "input" : input_data,
                    "output": sheets[0].range('L24').value
                }
            )
        for entry in output:
            input_data = entry['input'][provider]
            output_value = entry['output']
            record = {
                'carpan_brand_name': input_data['org_brand_name'].strip().replace('\xa0', ' '),
                'carpan_car_name': input_data['org_car_name'].strip().replace('\xa0', ' '),
                'carpan_model_name': input_data['org_model_name'].strip().replace('\xa0', ' '),
                'carpan_model_detail': input_data['org_model_detail'].strip().replace('\xa0', ' '),
                'provider' : provider,
                'brand_name': input_data['brand_name'][2],
                'car_name': input_data['car_name'][2],
                'model_name': input_data['model_name'][2],
                'car_price': input_data['car_price'][2],
                'deposit': input_data['deposit'][2] ,
                'downpayment': input_data['downpayment'][2] / input_data[provider]['car_price'][2],
                'period': input_data['period'][2],
                'output': output_value
            }
            records.append(record)
        df_out = pd.DataFrame(records)
        df_out['period'] = df_out['period'].apply(lambda x: convert_period(provider, x))
        df_out.to_csv(f"""./리스/결과/{record['provider']}_{record['carpan_brand_name']}_{record['carpan_car_name']}_{record['carpan_model_name'].replace('/', '')}_{idx}.csv""", encoding = 'cp949') 
        
        # if idx==10:
        #     break

0
1
2
3
4
5
6
7
8
9
10


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x107d5fdd0>>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 790, in _clean_thread_parent_frames
    active_threads = {thread.ident for thread in threading.enumerate()}
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 790, in <setcomp>
    active_threads = {thread.ident for thread in threading.enumerate()}
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt: 


KeyboardInterrupt: 

In [94]:
wb.close()

In [141]:
xl_app = xw.App(visible=True)
wb = xl_app.books.open('./리스/견적기/nh.xlsx')

In [142]:
def brand_idx(sh, x):
    x = x.strip().replace('\xa0', ' ')
    models = sh.range('I43', 'I79').value 
    models = [str(x).strip().replace('\xa0', ' ') for x in models]
    for idx, model in enumerate(models):
        if model == x:
            model_code = model[0]
            break
    return idx+1

def car_idx(sh,x):
    x = x.strip().replace('\xa0', ' ')
    models = sh.range('BZ3', 'BZ40').value 
    models = [str(x).strip().replace('\xa0', ' ') for x in models]
    for idx, model in enumerate(models):
        if model == x:
            model_code = model[0]
            break
    return idx+1

def car_model_idx(sh,x):
    x = str(x).strip().replace('\xa0', ' ')
    models = sh.range('CL4', 'CL30').value 
    models = [str(x).strip().replace('\xa0', ' ') for x in models]
    for idx, model in enumerate(models):
        if model == x:
            model_code = model[0]
            break
    return idx+1

In [143]:
sheet_names = ['운용리스', '1']
sheets = [wb.sheets[i] for i in sheet_names]
input_datum = df.apply(lambda x: convert_params('nh', x['brand_nh'], x['car_nh'], x['model_nh'], x['car_price'], x['brand_name'], x['car_name'], x['model_name'], x['model_detial']), axis=1)

In [149]:
deposits = [0, 0.1, 0.2, 0.3, 0.4]
downpayments = [0, 0.1, 0.2, 0.3, 0.4] 
periods = [60,48,36] # 60, 48, 36 
provider = 'nh'


for idx, input_data in enumerate(input_datum):
    print(idx)
    if input_data[provider]['car_name'][2]:
        inputs = []  
        for combo in itertools.product(deposits, downpayments, periods):
            modified_input = copy.deepcopy(input_data)
            modified_input[provider]['deposit'][-1] = combo[0]
            modified_input[provider]['downpayment'][-1] = combo[1] * input_data[provider]['car_price'][2]
            modified_input[provider]['period'][-1] = combo[2]
            inputs.append(modified_input)
    
        output = []
        records = []
        
        for input_data in inputs:
            for i in input_data[provider].items():
                if 'org' not in i[0]:
                    val = i[1][2]
                    if i[0] == 'brand_name':
                        # d = val
                        d = brand_idx(sheets[1], input_data['nh']['brand_name'][2])
                    elif i[0] == 'car_name':
                        d = car_idx(sheets[0], input_data['nh']['car_name'][2])
                    elif i[0] == 'model_name':
                        d = car_model_idx(sheets[0], input_data['nh']['model_name'][2])        
                    else:
                        d = i[1][2]
                    sheets[i[1][0]].range(i[1][1]).value = d 
            output.append(
                {
                    "input" : input_data,
                    "output": sheets[0].range('AG22').value
                }
            )
        for entry in output:
            input_data = entry['input'][provider]
            output_value = entry['output']
            record = {
                'carpan_brand_name': input_data['org_brand_name'].strip().replace('\xa0', ' '),
                'carpan_car_name': input_data['org_car_name'].strip().replace('\xa0', ' '),
                'carpan_model_name': input_data['org_model_name'].strip().replace('\xa0', ' '),
                'carpan_model_detail': input_data['org_model_detail'].strip().replace('\xa0', ' '),
                'provider' : provider,
                'brand_name': input_data['brand_name'][2],
                'car_name': input_data['car_name'][2],
                'model_name': input_data['model_name'][2],
                'car_price': input_data['car_price'][2],
                'deposit': input_data['deposit'][2] ,
                'downpayment': input_data['downpayment'][2] / input_data['car_price'][2],
                'period': input_data['period'][2],
                'output': output_value
            }
            records.append(record)
        df_out = pd.DataFrame(records)
        df_out['period'] = df_out['period'].apply(lambda x: convert_period(provider, x))
        df_out.to_csv(f"""./리스/결과/{record['provider']}_{record['carpan_brand_name']}_{record['carpan_car_name']}_{record['carpan_model_name'].replace('/', '')}_{idx}.csv""", encoding = 'cp949') 
        
        # if idx==10:
        #     break

0
1
2
3
4
5
6
7
8
9
10
11
12
13


KeyboardInterrupt: 

In [109]:
wb.close()